# CORTEX — Stage I: Beam Angle Prediction

> **Conv Front-End · RoPE · Causal Transformer · Circular Regression**

This notebook trains **CORTEX Stage I**, which predicts optimal TX/RX beam angles for THz UAV communications from mobility time-series features.  
It outputs two normalized 3-D unit vectors (one per antenna) and optimises a **Wrapped Cauchy loss** that handles circular angle geometry correctly.

### Architecture
```
Input [B, T, F]
  → Conv Front-End  (2× strided Conv1d + GELU + BN)
  → Causal Transformer  (N× RoPE self-attention + FFN)
  → MLP head  → 6-D output → L2-normalise → TX vec (3D) | RX vec (3D)
```

---
**Outputs saved to Google Drive** (optional) or local `outputs/` folder.


In [ ]:
# Install / upgrade required packages
!pip install torch pandas numpy matplotlib tqdm scikit-learn -q

In [ ]:
import os, glob, math, json, warnings
from datetime import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tqdm import tqdm

warnings.filterwarnings("ignore")
torch.manual_seed(42)
np.random.seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

## 1 · Setup: Output Directories

Run **either** the Google Colab block (mounts Drive) **or** the local block below.  
Skip whichever doesn't apply to your environment.

In [ ]:
# ── Option A: Google Colab + Drive ──────────────────────────────────────────
# Comment out this block if running locally.

try:
    from google.colab import drive
    drive.mount("/content/drive", force_remount=False)
    OUTPUT_ROOT = "/content/drive/MyDrive/CORTEX_Stage1"
    print("✅ Google Drive mounted")
except ImportError:
    # ── Option B: Local ──────────────────────────────────────────────────────
    OUTPUT_ROOT = "outputs"
    print("ℹ️  Running locally — outputs go to ./outputs/")

MODELS_DIR  = os.path.join(OUTPUT_ROOT, "models")
RESULTS_DIR = os.path.join(OUTPUT_ROOT, "results")
PLOTS_DIR   = os.path.join(OUTPUT_ROOT, "plots")

for d in [MODELS_DIR, RESULTS_DIR, PLOTS_DIR]:
    os.makedirs(d, exist_ok=True)

print(f"Models  → {MODELS_DIR}")
print(f"Results → {RESULTS_DIR}")
print(f"Plots   → {PLOTS_DIR}")

## 2 · Load Data

Point `CSV_PATH` to your mobility CSV file.

In [ ]:
# ── Set your CSV path here ───────────────────────────────────────────────────
CSV_PATH = "data/mobility.csv"          # <-- change this

# Auto-detect if not found at the explicit path
if not os.path.exists(CSV_PATH):
    # Look in Google Drive common locations
    for candidate in glob.glob("/content/drive/**/*.csv", recursive=True):
        CSV_PATH = candidate
        print(f"Auto-detected: {CSV_PATH}")
        break
    else:
        # Colab file upload fallback
        try:
            from google.colab import files
            print("No CSV found — please upload your file:")
            uploaded = files.upload()
            CSV_PATH = list(uploaded.keys())[0]
        except ImportError:
            raise FileNotFoundError(
                f"CSV not found at '{CSV_PATH}'. "
                "Set CSV_PATH to the correct path."
            )

data = pd.read_csv(CSV_PATH)
print(f"\nLoaded {len(data):,} rows | {data['scenario'].nunique():,} scenarios")
print(f"Columns: {list(data.columns)}")

# Per-motion summary
print(f"\n{'Motion Type':<20} {'Samples':>10} {'Scenarios':>10}")
print("-" * 45)
for mt in sorted(data["motion_type"].unique()):
    md = data[data["motion_type"] == mt]
    print(f"{mt:<20} {len(md):>10,} {md['scenario'].nunique():>10}")

## 3 · Circular Angle Utilities

In [ ]:
def angles_to_unit_vectors(phi, theta):
    """(phi, theta) in degrees → 3-D unit vectors."""
    phi_r, theta_r = np.deg2rad(phi), np.deg2rad(theta)
    return np.stack([
        np.sin(theta_r) * np.cos(phi_r),
        np.sin(theta_r) * np.sin(phi_r),
        np.cos(theta_r),
    ], axis=-1)


def unit_vectors_to_angles(vecs):
    """3-D unit vectors → (phi, theta) in degrees."""
    x, y, z = vecs[..., 0], vecs[..., 1], vecs[..., 2]
    theta = np.rad2deg(np.arccos(np.clip(z, -1, 1)))
    phi   = np.rad2deg(np.arctan2(y, x))
    phi   = np.where(phi < 0, phi + 360, phi)
    return phi, theta


def circular_mae(pred, true):
    """Circular mean absolute error (degrees, wraps at 360°)."""
    diff = np.abs(pred - true)
    diff = np.where(diff > 180, 360 - diff, diff)
    return float(np.mean(diff))

## 4 · Dataset

In [ ]:
FEATURE_COLS = [
    "time", "scenario", "motion_type_encoded", "distance",
    "velocity", "acceleration", "los_angle_tx", "los_angle_rx",
    "los_ele_tx", "los_ele_rx", "velocity_angle", "mobility_factor",
]


class MobilityTimeSeriesDataset(Dataset):
    def __init__(self, data, sequence_length=10,
                 scaler=None, label_encoder=None, fit=True):
        self.sequence_length = sequence_length
        data = data.copy()

        if label_encoder is None:
            self.label_encoder = LabelEncoder()
            data["motion_type_encoded"] = self.label_encoder.fit_transform(data["motion_type"])
        else:
            self.label_encoder = label_encoder
            data["motion_type_encoded"] = self.label_encoder.transform(data["motion_type"])

        # Targets → 6-D unit vectors
        vec_tx = angles_to_unit_vectors(
            data["phi_tx_optimal"].values, data["theta_tx_optimal"].values)
        vec_rx = angles_to_unit_vectors(
            data["phi_rx_optimal"].values, data["theta_rx_optimal"].values)
        target_vectors = np.concatenate([vec_tx, vec_rx], axis=-1)

        self.original_angles = np.stack([
            data["phi_tx_optimal"].values, data["theta_tx_optimal"].values,
            data["phi_rx_optimal"].values, data["theta_rx_optimal"].values,
        ], axis=-1)

        if scaler is None:
            self.scaler = StandardScaler()
            features_scaled = self.scaler.fit_transform(data[FEATURE_COLS])
        else:
            self.scaler = scaler
            features_scaled = self.scaler.transform(data[FEATURE_COLS])

        # Sliding windows per scenario
        self.sequences, self.targets, self.target_angles = [], [], []
        data_idx = data.reset_index(drop=True)
        feat_df  = pd.DataFrame(features_scaled, index=data_idx.index)

        for scenario in tqdm(data_idx["scenario"].unique(),
                             desc="Creating sequences", leave=False):
            idx = (data_idx[data_idx["scenario"] == scenario]
                   .sort_values("time").index)
            if len(idx) < sequence_length + 1:
                continue
            feat = feat_df.loc[idx].values
            tgts = target_vectors[idx]
            angs = self.original_angles[idx]
            for i in range(len(idx) - sequence_length):
                self.sequences.append(feat[i : i + sequence_length])
                self.targets.append(tgts[i + sequence_length])
                self.target_angles.append(angs[i + sequence_length])

        self.sequences    = np.array(self.sequences,    dtype=np.float32)
        self.targets      = np.array(self.targets,      dtype=np.float32)
        self.target_angles= np.array(self.target_angles,dtype=np.float32)
        print(f"  Created {len(self.sequences):,} sequences")

    def __len__(self):  return len(self.sequences)
    def __getitem__(self, idx):
        return (torch.FloatTensor(self.sequences[idx]),
                torch.FloatTensor(self.targets[idx]))

## 5 · Stratified Train / Val / Test Split

In [ ]:
def stratified_split(data, train_ratio=0.70, val_ratio=0.15,
                      test_ratio=0.15, random_state=42):
    rng = np.random.default_rng(random_state)
    train_sc, val_sc, test_sc = [], [], []
    print(f"{'Motion type':<20} {'total':>6} {'train':>6} {'val':>6} {'test':>6}")
    print("-" * 48)
    for mt in sorted(data["motion_type"].unique()):
        sc = data[data["motion_type"] == mt]["scenario"].unique().copy()
        rng.shuffle(sc)
        n_tr = int(train_ratio * len(sc))
        n_va = int(val_ratio   * len(sc))
        train_sc.extend(sc[:n_tr])
        val_sc.extend(sc[n_tr : n_tr + n_va])
        test_sc.extend(sc[n_tr + n_va :])
        print(f"  {mt:<18} {len(sc):>6} {n_tr:>6} {n_va:>6} "
              f"{len(sc)-n_tr-n_va:>6}")
    return train_sc, val_sc, test_sc

## 6 · Model — Conv Front-End + RoPE Causal Transformer

In [ ]:
# ── RoPE helpers ─────────────────────────────────────────────────────────────

def _build_rope_cache(seq_len, head_dim, base=10000.0, device="cpu", dtype=torch.float32):
    theta = 1.0 / (base ** (torch.arange(0, head_dim, 2, device=device, dtype=dtype) / head_dim))
    t = torch.arange(seq_len, device=device, dtype=dtype)
    freqs = torch.einsum("t,f->tf", t, theta)
    return torch.cos(freqs), torch.sin(freqs)


def _apply_rope(q, k, cos, sin):
    q1, q2 = q[..., ::2], q[..., 1::2]
    k1, k2 = k[..., ::2], k[..., 1::2]
    cos = cos.unsqueeze(0).unsqueeze(2)
    sin = sin.unsqueeze(0).unsqueeze(2)
    q_rot = torch.stack([q1*cos - q2*sin, q1*sin + q2*cos], dim=-1).flatten(-2)
    k_rot = torch.stack([k1*cos - k2*sin, k1*sin + k2*cos], dim=-1).flatten(-2)
    return q_rot, k_rot


def _causal_mask(T, device):
    return torch.triu(torch.ones(T, T, device=device, dtype=torch.bool), diagonal=1)


# ── Sub-modules ───────────────────────────────────────────────────────────────

class ConvFrontEnd(nn.Module):
    def __init__(self, in_dim, d_model, dropout=0.1):
        super().__init__()
        self.proj = nn.Linear(in_dim, d_model)
        self.conv = nn.Sequential(
            nn.Conv1d(d_model, d_model, kernel_size=5, stride=2, padding=2),
            nn.GELU(), nn.BatchNorm1d(d_model),
            nn.Conv1d(d_model, d_model, kernel_size=3, stride=2, padding=1),
            nn.GELU(), nn.BatchNorm1d(d_model),
        )
        self.drop = nn.Dropout(dropout)

    def forward(self, x):
        h = self.proj(x).transpose(1, 2)
        return self.drop(self.conv(h).transpose(1, 2))


class RoPEAttention(nn.Module):
    def __init__(self, d_model, n_heads, dropout=0.1):
        super().__init__()
        self.n_heads  = n_heads
        self.head_dim = d_model // n_heads
        self.qkv = nn.Linear(d_model, 3 * d_model)
        self.out = nn.Linear(d_model, d_model)
        self.drop = nn.Dropout(dropout)

    def forward(self, x, attn_mask=None):
        B, T, D = x.shape
        q, k, v = self.qkv(x).chunk(3, dim=-1)
        reshape = lambda z: z.view(B, T, self.n_heads, self.head_dim)
        q, k, v = reshape(q), reshape(k), reshape(v)
        cos, sin = _build_rope_cache(T, self.head_dim, device=x.device, dtype=x.dtype)
        q, k = _apply_rope(q, k, cos, sin)
        scores = torch.einsum("bthd,bThd->bhtT", q, k) / math.sqrt(self.head_dim)
        if attn_mask is not None:
            scores = scores.masked_fill(attn_mask.unsqueeze(0).unsqueeze(0), float("-inf"))
        attn = self.drop(torch.softmax(scores, dim=-1))
        y = torch.einsum("bhtT,bThd->bthd", attn, v).contiguous().view(B, T, D)
        return self.out(y)


class TransformerBlock(nn.Module):
    def __init__(self, d_model, n_heads, dim_ff, dropout=0.1):
        super().__init__()
        self.attn = RoPEAttention(d_model, n_heads, dropout)
        self.ln1  = nn.LayerNorm(d_model)
        self.ff   = nn.Sequential(
            nn.Linear(d_model, dim_ff), nn.GELU(), nn.Dropout(dropout),
            nn.Linear(dim_ff, d_model), nn.Dropout(dropout),
        )
        self.ln2  = nn.LayerNorm(d_model)

    def forward(self, x, attn_mask=None):
        x = self.ln1(x + self.attn(x, attn_mask))
        x = self.ln2(x + self.ff(x))
        return x


# ── Top-level model ───────────────────────────────────────────────────────────

class CORTEXAnglePredictor(nn.Module):
    """
    CORTEX Stage I.
    Input : [B, T, F]
    Output: [B, 6]  — two normalised 3-D unit vectors (TX, RX)
    """
    def __init__(self, input_size, d_model=256, n_heads=8, n_layers=4,
                 dim_feedforward=512, dropout=0.2):
        super().__init__()
        self.frontend = ConvFrontEnd(input_size, d_model, dropout)
        self.blocks   = nn.ModuleList([
            TransformerBlock(d_model, n_heads, dim_feedforward, dropout)
            for _ in range(n_layers)
        ])
        self.fuse = nn.Linear(d_model, d_model)
        self.head = nn.Sequential(
            nn.Linear(d_model, d_model), nn.LayerNorm(d_model),
            nn.GELU(), nn.Dropout(dropout),
            nn.Linear(d_model, d_model // 2), nn.LayerNorm(d_model // 2),
            nn.GELU(), nn.Dropout(dropout),
            nn.Linear(d_model // 2, 6),
        )

    def forward(self, x):
        h = self.frontend(x)
        causal = _causal_mask(h.size(1), h.device)
        for blk in self.blocks:
            h = blk(h, causal)
        last  = h[:, -1, :]
        fused = last + 0.5 * self.fuse(h.mean(dim=1))
        out   = self.head(fused)
        tx, rx = out[:, :3], out[:, 3:]
        tx = tx / (tx.norm(dim=1, keepdim=True) + 1e-8)
        rx = rx / (rx.norm(dim=1, keepdim=True) + 1e-8)
        return torch.cat([tx, rx], dim=1)

## 7 · Wrapped Cauchy Loss

In [ ]:
class WrappedCauchyLoss(nn.Module):
    """
    Circular-aware loss for paired 3-D unit vectors (TX + RX).
    Uses the negative log-likelihood of a Wrapped Cauchy distribution
    computed via cosine similarity.

    Parameters
    ----------
    rho : float   Concentration ∈ (0,1). Higher = sharper.
    """
    def __init__(self, rho=0.9, eps=1e-8):
        super().__init__()
        self.rho, self.eps = rho, eps

    def forward(self, pred, target):
        pred_tx,  pred_rx  = pred[:, :3],   pred[:, 3:]
        tgt_tx,   tgt_rx   = target[:, :3], target[:, 3:]
        cos_tx = torch.sum(pred_tx * tgt_tx, dim=1).clamp(-1+self.eps, 1-self.eps)
        cos_rx = torch.sum(pred_rx * tgt_rx, dim=1).clamp(-1+self.eps, 1-self.eps)
        rho2   = self.rho ** 2
        return torch.mean(
            torch.log(1 + rho2 - 2*self.rho*cos_tx + self.eps) +
            torch.log(1 + rho2 - 2*self.rho*cos_rx + self.eps)
        )

## 8 · Hyperparameters

In [ ]:
# ── Data ─────────────────────────────────────────────────────────────────────
SEQUENCE_LENGTH = 15
BATCH_SIZE      = 512

# ── Model ─────────────────────────────────────────────────────────────────────
D_MODEL    = 256
N_HEADS    = 8
N_LAYERS   = 4
DIM_FF     = 512
DROPOUT    = 0.2

# ── Training ──────────────────────────────────────────────────────────────────
LEARNING_RATE = 1e-3
NUM_EPOCHS    = 100
PATIENCE      = None    # Set an integer to enable early stopping, e.g. 15

print("Hyperparameters set.")

## 9 · Build Datasets & DataLoaders

In [ ]:
train_sc, val_sc, test_sc = stratified_split(data)

train_data = data[data["scenario"].isin(train_sc)].reset_index(drop=True)
val_data   = data[data["scenario"].isin(val_sc)].reset_index(drop=True)
test_data  = data[data["scenario"].isin(test_sc)].reset_index(drop=True)

print("\nBuilding datasets...")
train_ds = MobilityTimeSeriesDataset(train_data, SEQUENCE_LENGTH)
val_ds   = MobilityTimeSeriesDataset(val_data,   SEQUENCE_LENGTH,
               scaler=train_ds.scaler, label_encoder=train_ds.label_encoder)
test_ds  = MobilityTimeSeriesDataset(test_data,  SEQUENCE_LENGTH,
               scaler=train_ds.scaler, label_encoder=train_ds.label_encoder)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,
                          num_workers=2, pin_memory=True)
val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False,
                          num_workers=2, pin_memory=True)
test_loader  = DataLoader(test_ds,  batch_size=BATCH_SIZE, shuffle=False,
                          num_workers=2, pin_memory=True)

print(f"\nTrain: {len(train_ds):,}  |  Val: {len(val_ds):,}  |  Test: {len(test_ds):,}")

## 10 · Instantiate Model

In [ ]:
input_size = train_ds.sequences.shape[2]

model = CORTEXAnglePredictor(
    input_size=input_size,
    d_model=D_MODEL, n_heads=N_HEADS, n_layers=N_LAYERS,
    dim_feedforward=DIM_FF, dropout=DROPOUT,
).to(device)

total_p = sum(p.numel() for p in model.parameters())
print(f"Parameters : {total_p:,}  ({total_p*4/1024/1024:.1f} MB)")
print(f"Input size : {input_size} features  |  Output: 6-D (2 × 3-D unit vectors)")

## 11 · Training Loop

In [ ]:
def run_epoch(model, loader, optimizer, criterion, training=True):
    model.train() if training else model.eval()
    total = 0.0
    ctx = torch.enable_grad() if training else torch.no_grad()
    with ctx:
        for seqs, tgts in tqdm(loader, desc="  train" if training else "  val  ",
                               leave=False):
            seqs, tgts = seqs.to(device), tgts.to(device)
            preds = model(seqs)
            loss  = criterion(preds, tgts)
            if training:
                optimizer.zero_grad(set_to_none=True)
                loss.backward()
                nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                optimizer.step()
            total += float(loss.detach())
    return total / max(1, len(loader))


criterion = WrappedCauchyLoss(rho=0.9)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode="min", factor=0.5, patience=5)

train_losses, val_losses = [], []
best_val    = float("inf")
best_ckpt   = os.path.join(MODELS_DIR, "stage1_best_model.pth")
no_improve  = 0

for epoch in range(NUM_EPOCHS):
    print(f"\nEpoch {epoch+1}/{NUM_EPOCHS}")
    print("-" * 50)

    tr = run_epoch(model, train_loader, optimizer, criterion, training=True)
    va = run_epoch(model, val_loader,   optimizer, criterion, training=False)

    old_lr = optimizer.param_groups[0]["lr"]
    scheduler.step(va)
    new_lr = optimizer.param_groups[0]["lr"]
    if new_lr < old_lr:
        print(f"  📉 LR {old_lr:.2e} → {new_lr:.2e}")

    train_losses.append(tr)
    val_losses.append(va)
    print(f"  Train: {tr:.6f}  |  Val: {va:.6f}")

    if va < best_val:
        best_val  = va
        no_improve = 0
        torch.save(model.state_dict(), best_ckpt)
        print(f"  ✅ Best saved (val={va:.6f})")
    else:
        no_improve += 1
        if PATIENCE and no_improve >= PATIENCE:
            print(f"\n⚠️  Early stopping at epoch {epoch+1}")
            break

print(f"\nTraining complete. Best val loss: {best_val:.6f}")

In [ ]:
# Load the best checkpoint before evaluation
model.load_state_dict(torch.load(best_ckpt, map_location=device))
print(f"Best model loaded from: {best_ckpt}")

## 12 · Evaluation

In [ ]:
@torch.no_grad()
def evaluate(model, loader):
    model.eval()
    all_preds, all_tgts = [], []
    for seqs, tgts in tqdm(loader, desc="  eval", leave=False):
        all_preds.append(model(seqs.to(device)).cpu().numpy())
        all_tgts.append(tgts.numpy())

    preds = np.vstack(all_preds)
    tgts  = np.vstack(all_tgts)

    pred_phi_tx,   pred_theta_tx  = unit_vectors_to_angles(preds[:, :3])
    pred_phi_rx,   pred_theta_rx  = unit_vectors_to_angles(preds[:, 3:])
    true_phi_tx,   true_theta_tx  = unit_vectors_to_angles(tgts[:, :3])
    true_phi_rx,   true_theta_rx  = unit_vectors_to_angles(tgts[:, 3:])

    pred_ang = np.stack([pred_phi_tx, pred_theta_tx, pred_phi_rx, pred_theta_rx], -1)
    true_ang = np.stack([true_phi_tx, true_theta_tx, true_phi_rx, true_theta_rx], -1)

    mae  = np.array([
        circular_mae(pred_phi_tx,  true_phi_tx),
        float(np.mean(np.abs(pred_theta_tx - true_theta_tx))),
        circular_mae(pred_phi_rx,  true_phi_rx),
        float(np.mean(np.abs(pred_theta_rx - true_theta_rx))),
    ])
    diff = np.abs(pred_ang - true_ang)
    diff = np.where(diff > 180, 360 - diff, diff)
    mse  = np.mean(diff**2, axis=0)
    rmse = np.sqrt(mse)
    ss_r = np.sum((true_ang - pred_ang)**2, axis=0)
    ss_t = np.sum((true_ang - true_ang.mean(0))**2, axis=0)
    r2   = 1 - ss_r / (ss_t + 1e-8)

    ang_tx = np.rad2deg(np.arccos(np.clip(
        np.sum(preds[:, :3] * tgts[:, :3], axis=1), -1, 1)))
    ang_rx = np.rad2deg(np.arccos(np.clip(
        np.sum(preds[:, 3:] * tgts[:, 3:], axis=1), -1, 1)))

    names = ["phi_tx", "theta_tx", "phi_rx", "theta_rx"]
    print("\n" + "="*70)
    print("EVALUATION RESULTS")
    print("="*70)
    print(f"  {'Angle':<16} {'MSE':>10} {'RMSE':>10} {'MAE (°)':>10} {'R²':>10}")
    print("  " + "-"*50)
    for i, n in enumerate(names):
        print(f"  {n:<16} {mse[i]:10.4f} {rmse[i]:10.4f} {mae[i]:10.4f} {r2[i]:10.4f}")
    print("="*70)
    print(f"  Angular dist TX : {ang_tx.mean():.4f}° ± {ang_tx.std():.4f}°")
    print(f"  Angular dist RX : {ang_rx.mean():.4f}° ± {ang_rx.std():.4f}°")
    print(f"  Overall MAE     : {mae.mean():.4f}°")
    print(f"  Overall R²      : {r2.mean():.4f}")
    print("="*70)

    return pred_ang, true_ang, mse, mae, r2

predictions, targets_eval, mse, mae, r2 = evaluate(model, test_loader)

## 13 · Visualisations

In [ ]:
ANGLE_NAMES = ["phi_tx_optimal", "theta_tx_optimal",
               "phi_rx_optimal", "theta_rx_optimal"]

fig = plt.figure(figsize=(16, 10))

# Loss curve
ax = fig.add_subplot(3, 3, 1)
ax.plot(train_losses, label="Train", lw=2)
ax.plot(val_losses,   label="Val",   lw=2)
ax.set(xlabel="Epoch", ylabel="Loss",
       title="Training & Validation Loss")
ax.set_yscale("log"); ax.legend(); ax.grid(alpha=0.3)

# Scatter: predicted vs true
idx = np.random.choice(len(predictions), min(1000, len(predictions)), replace=False)
for i in range(4):
    ax = fig.add_subplot(3, 3, i + 2)
    ax.scatter(targets_eval[idx, i], predictions[idx, i],
               alpha=0.5, s=15, edgecolors="none")
    lo = min(targets_eval[:, i].min(), predictions[:, i].min())
    hi = max(targets_eval[:, i].max(), predictions[:, i].max())
    ax.plot([lo, hi], [lo, hi], "r--", lw=2, label="Perfect")
    ax.set(xlabel="True (°)", ylabel="Predicted (°)",
           title=ANGLE_NAMES[i])
    ax.legend(fontsize=9); ax.grid(alpha=0.3)

# Error histograms
for i in range(4):
    ax = fig.add_subplot(3, 3, i + 6)
    errs = predictions[:, i] - targets_eval[:, i]
    if "phi" in ANGLE_NAMES[i]:
        errs = (errs + 180) % 360 - 180
    ax.hist(errs, bins=50, alpha=0.7, edgecolor="black")
    ax.axvline(0, color="red", linestyle="--", lw=2)
    ax.set(xlabel="Error (°)", ylabel="Frequency",
           title=f"{ANGLE_NAMES[i]} — Error Dist.")
    ax.grid(alpha=0.3)

plt.tight_layout()
plot_path = os.path.join(PLOTS_DIR, "stage1_training_results.png")
plt.savefig(plot_path, dpi=150, bbox_inches="tight")
plt.show()
print(f"Plot saved → {plot_path}")

## 14 · Save Final Model & Metrics

In [ ]:
ts = datetime.now().strftime("%Y%m%d_%H%M%S")

# Full checkpoint (model + preprocessing objects + metadata)
final_ckpt = os.path.join(MODELS_DIR, f"stage1_model_{ts}.pth")
torch.save({
    "model_state_dict": model.state_dict(),
    "scaler":           train_ds.scaler,
    "label_encoder":    train_ds.label_encoder,
    "hyperparameters": {
        "input_size": input_size,
        "sequence_length": SEQUENCE_LENGTH,
        "d_model": D_MODEL, "n_heads": N_HEADS,
        "n_layers": N_LAYERS, "dim_feedforward": DIM_FF,
        "dropout": DROPOUT,
    },
    "train_losses": train_losses,
    "val_losses":   val_losses,
    "test_metrics": {
        "mae":  mae.tolist(),
        "mse":  mse.tolist(),
        "r2":   r2.tolist(),
    },
    "timestamp": ts,
}, final_ckpt)

# Human-readable metrics JSON
metrics_path = os.path.join(RESULTS_DIR, f"stage1_metrics_{ts}.json")
with open(metrics_path, "w") as f:
    json.dump({
        "overall_mae":  float(mae.mean()),
        "overall_r2":   float(r2.mean()),
        "best_val_loss": float(min(val_losses)),
        "mae_per_angle": dict(zip(
            ["phi_tx", "theta_tx", "phi_rx", "theta_rx"],
            [float(x) for x in mae])),
        "r2_per_angle":  dict(zip(
            ["phi_tx", "theta_tx", "phi_rx", "theta_rx"],
            [float(x) for x in r2])),
        "timestamp": ts,
    }, f, indent=2)

print(f"Model   → {final_ckpt}")
print(f"Metrics → {metrics_path}")
print(f"\nBest val loss : {min(val_losses):.6f}")
print(f"Test MAE      : {mae.mean():.4f}°")
print(f"Test R²       : {r2.mean():.4f}")

## 15 · Loading a Checkpoint (for Stage II)

Use this snippet to reload the model in another notebook or script.

In [ ]:
# Example: reload the saved model
ckpt = torch.load(final_ckpt, map_location=device)
hp   = ckpt["hyperparameters"]

loaded_model = CORTEXAnglePredictor(
    input_size     = hp["input_size"],
    d_model        = hp["d_model"],
    n_heads        = hp["n_heads"],
    n_layers       = hp["n_layers"],
    dim_feedforward= hp["dim_feedforward"],
    dropout        = hp["dropout"],
).to(device)

loaded_model.load_state_dict(ckpt["model_state_dict"])
loaded_model.eval()

scaler        = ckpt["scaler"]          # StandardScaler — pass to val/test datasets
label_encoder = ckpt["label_encoder"]  # LabelEncoder   — pass to val/test datasets

print("Model reloaded successfully.")
print(f"Sequence length: {hp['sequence_length']}")
print(f"Input size:      {hp['input_size']}")